# Import Package

In [12]:
# Import packages
import os
import cv2
import numpy as np
import tensorflow as tf
import sys

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

from IPython.display import clear_output

# Import utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

In [13]:
# Name of the directory containing the object detection module we're using
#MODEL_NAME = 'inference_graph'
#MODEL_NAME = 'faster_rcnn_inception_v2_coco_2018_01_28'
MODEL_NAME = 'ssd_mobilenet_v2_coco_2018_03_29'

# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'mscoco_label_map.pbtxt')

# Number of classes the object detector can identify
NUM_CLASSES = 90

In [14]:
## Load the label map.
# Label maps map indices to category names, so that when our convolution
# network predicts `5`, we know that this corresponds to `king`.
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

In [17]:
# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

# Initialize webcam feed
video = cv2.VideoCapture(0)
ret = video.set(3,720)
ret = video.set(4,480)

def count_person(idx,idx_score):
    os.system('mode con: cols=50 lines=9')
    idx_detected = idx[0][idx_score[0] > 0.85]
    idx_person = idx_detected[idx_detected == 1]
    idx_bicycle = idx_detected[idx_detected == 2]
    idx_car = idx_detected[idx_detected == 3]
    idx_motorcycle = idx_detected[idx_detected == 4]
    idx_bus = idx_detected[idx_detected == 6]
    idx_truck = idx_detected[idx_detected == 8]
    idx_chair = idx_detected[idx_detected == 62]
    clear_output(wait=True)
    print('======================')
    print('Person Count     : ',idx_person.size)
    print('Chair Count      : ',idx_chair.size)	
    print('Bicycle Count    : ',idx_bicycle.size)
    print('Car Count        : ',idx_car.size)
    print('Motorcycle Count : ',idx_motorcycle.size)
    print('Bus Count        : ',idx_bus.size)
    print('Truck Count      : ',idx_truck.size)

In [18]:
while(True):

    # Acquire frame and expand frame dimensions to have shape: [1, None, None, 3]
    # i.e. a single-column array, where each item in the column has the pixel RGB value
    ret, frame = video.read()
    frame_expanded = np.expand_dims(frame, axis=0)

    # Perform the actual detection by running the model with the image as input
    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: frame_expanded})
	
	# Counting Object Detected
    count_person(classes,scores)
		
    # Draw the results of the detection (aka 'visulaize the results')
    vis_util.visualize_boxes_and_labels_on_image_array(
        frame,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8,
        min_score_thresh=0.85)

    # All the results have been drawn on the frame, so it's time to display it.
    cv2.imshow('Object detector', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) == ord('q'):
        os.system('mode con: cols=120 lines=20')		
        break

# Clean up
video.release()
cv2.destroyAllWindows()

Person Count     :  1
Chair Count      :  0
Bicycle Count    :  0
Car Count        :  0
Motorcycle Count :  0
Bus Count        :  0
Truck Count      :  0
